In [14]:
import pandas as pd
import sqlite3
import plotly.express as px
from plotly.subplots import make_subplots

date_parser = {
    "StudyStartDate": {
        "format": r"%y/%m/%d"
    },
    "StudyCompletionDate": {
        "format": r"%y/%m/%d"
    },
    "LastUpdatePostDate": {
        "format": r"%y/%m/%d"
    }
}

In [20]:
conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')

query = """
SELECT 
    COUNT(i.Intervention) as InterventionCount, m.nctID, i.Intervention, i.InterventionType
FROM 
    CTGMain as m 
LEFT JOIN 
    CTGInterventions as i 
    ON m.nctID=i.nctID 
WHERE 
    StudyType='interventional'
AND 
    PrimaryPurpose='treatment'
AND
    InterventionType='drug'
GROUP BY 
    i.Intervention 
ORDER BY 
    InterventionCount desc;"""

meshquery = """
SELECT 
    COUNT(mi.MeshIntervention) as MeshInterventionCount, m.nctID, mi.MeshIntervention, i.InterventionType
FROM 
    CTGMain as m 
LEFT JOIN 
    CTGMeshInterventions as mi 
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
WHERE 
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    i.InterventionType='drug'
GROUP BY 
    mi.MeshIntervention 
ORDER BY 
    MeshInterventionCount desc;"""

interventional_df = pd.read_sql_query(query, conn, parse_dates=date_parser)
mesh_interventional_df = pd.read_sql_query(meshquery, conn, parse_dates=date_parser)
conn.close()

print(interventional_df[-10:-1])
print(mesh_interventional_df)



      InterventionCount        nctID  \
1715                  1  nct01513733   
1716                  1  nct06849544   
1717                  1  nct06849544   
1718                  1  nct07024030   
1719                  1  nct06849544   
1720                  1  nct06849544   
1721                  1  nct07024030   
1722                  1  nct03796195   
1723                  1  nct06705686   

                                           Intervention InterventionType  
1715                                             0.5 mg             drug  
1716  0.06mg/kg zopocianine given days prior to surgery             drug  
1717         0.06mg/kg zopocianine given day of surgery             drug  
1718                                          0.04mg/kg             drug  
1719   0.03mg/kg zopocianine given day prior to surgery             drug  
1720         0.03mg/kg zopocianine given day of surgery             drug  
1721                                          0.02mg/kg             drug  

In [ ]:
fig = px.bar(interventional_df[:10], x='Intervention', y='InterventionCount', 
             hover_data='InterventionCount', color='Intervention', 
             title='Ten Most Prevalent Prostate Cancer Interventions')
fig.update_layout(showlegend=False)
fig.show()

In [21]:
mesh_interventional_df = mesh_interventional_df[mesh_interventional_df['MeshIntervention'] != 'null']
fig = px.bar(mesh_interventional_df[:10], x='MeshIntervention', y='MeshInterventionCount', 
             hover_data='MeshInterventionCount', color='MeshIntervention', 
             title='Ten Most Prevalent Prostate Cancer MeSH Interventions')
fig.update_layout(showlegend=False)
fig.show()

Now we look at top MeSH interventions in last 5 years

In [67]:
date_parser = {
    "StudyStartDate": {
        "format": r"%y/%m/%d"
    },
    "StudyCompletionDate": {
        "format": r"%y/%m/%d"
    },
    "LastUpdatePostDate": {
        "format": r"%y/%m/%d"
    }
}

conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')
meshquery1 = """
SELECT 
    COUNT(mi.MeshIntervention) as MeshInterventionCount, m.*, mi.MeshIntervention, i.InterventionType
FROM
    CTGMain as m
LEFT JOIN
    CTGMeshInterventions as mi
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
WHERE
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    StudyStartDate >= '2020-01-01'
AND
    InterventionType='drug'
GROUP BY
    mi.MeshIntervention
ORDER BY
    MeshInterventionCount desc;
"""
mesh_5yr_interventional_df = pd.read_sql_query(meshquery1, conn, parse_dates=date_parser)
conn.close()

print(mesh_5yr_interventional_df)


     MeshInterventionCount        nctID  \
0                      299  nct06863272   
1                      246  nct03649841   
2                      242  nct06329830   
3                      183  nct05521412   
4                      169  nct04523207   
..                     ...          ...   
276                      1  nct06129851   
277                      1  nct05011188   
278                      1  nct05574712   
279                      1  nct05501548   
280                      1  nct04702737   

                                            BriefTitle      Acronym  \
0    a clinical study of ifinatamab deruxtecan base...         null   
1    antiandrogen therapy, abiraterone acetate, and...         null   
2    177lu-psma, niraparib/aa plus prednisone for p...       lunaar   
3    evaluation of radioligand treatment in men wit...       violet   
4    a study of apalutamide (adjuvant treatment) an...         null   
..                                                 ...   

In [ ]:
mesh_5yr_interventional_df = mesh_5yr_interventional_df[mesh_5yr_interventional_df['MeshIntervention'] != 'null']
fig = px.bar(mesh_5yr_interventional_df[:10], x='MeshIntervention', y='MeshInterventionCount', 
             hover_data='MeshInterventionCount', color='MeshIntervention', 
             title='Ten Most Prevalent Prostate Cancer Drug MeSH Interventions Since 2020')
fig.update_layout(showlegend=False)
fig.update_xaxes(automargin=True)
fig.show()

What about clinical trials that are

In [10]:
date_parser = {
    "StudyStartDate": {
        "format": r"%y/%m/%d"
    },
    "StudyCompletionDate": {
        "format": r"%y/%m/%d"
    },
    "LastUpdatePostDate": {
        "format": r"%y/%m/%d"
    }
}

conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')
meshquery1 = """
SELECT 
    m.*, mi.MeshIntervention, i.InterventionType, 
    COUNT(case p.Phase when 'early_phase1' then 1 else null end) as EPhase1Count,
    COUNT(case p.Phase when 'phase1' then 1 else null end) as Phase1Count,
    COUNT(case p.Phase when 'phase2' then 1 else null end) as Phase2Count,
    COUNT(case p.Phase when 'phase3' then 1 else null end) as Phase3Count,
    COUNT(case p.Phase when 'phase4' then 1 else null end) as Phase4Count,
    COUNT(mi.MeshIntervention) as MeshInterventionCount
FROM
    CTGMain as m
LEFT JOIN
    CTGMeshInterventions as mi
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
LEFT JOIN
    CTGPhases as p
    ON m.nctID=p.nctID
WHERE
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    StudyStartDate >= '2020-01-01'
AND
    InterventionType='drug'
GROUP BY
    mi.MeshIntervention
ORDER BY
    MeshInterventionCount desc;
"""
mesh_5yr_interventional_df = pd.read_sql_query(meshquery1, conn, parse_dates=date_parser)
conn.close()

print(mesh_5yr_interventional_df)


           nctID                                         BriefTitle  \
0    nct06863272  a clinical study of ifinatamab deruxtecan base...   
1    nct06329830  177lu-psma, niraparib/aa plus prednisone for p...   
2    nct03649841  antiandrogen therapy, abiraterone acetate, and...   
3    nct05521412  evaluation of radioligand treatment in men wit...   
4    nct04126070  nivolumab + docetaxel + adt in mhspc patients ...   
..           ...                                                ...   
276  nct06029998  bortezomib in patients with metastatic castrat...   
277  nct06129851  relugolix in combination with radiation therap...   
278  nct05574712  a neoadjuvant study of abiraterone acetate, le...   
279  nct05501548  phase ii study of parp inhibitor olaparib and ...   
280  nct04702737  a study of amg 757 in participants with neuroe...   

         Acronym                                            OrgName  \
0           null                            merck sharp & dohme llc   
1    

In [32]:
mesh_5yr_interventional_df = mesh_5yr_interventional_df[mesh_5yr_interventional_df['MeshIntervention'] != 'null']
mesh_5yr_interventional_df['MeshIntervention'] = mesh_5yr_interventional_df['MeshIntervention'].astype(str).apply(lambda x: x if len(x) <= 20 else x[:20] + '...')
mesh_5yr_interventional_df.rename(columns={'EPhase1Count': 'Early Phase 1', 'Phase1Count': 'Phase 1', 'Phase2Count': 'Phase 2', 'Phase3Count': 'Phase 3', 'Phase4Count': 'Phase 4'}, inplace=True)
fig = px.bar(mesh_5yr_interventional_df[:10], x='MeshIntervention', y=['Early Phase 1', 'Phase 1', 'Phase 2', 'Phase 3', 'Phase 4'], 
             hover_data='MeshInterventionCount',
             orientation='v', width=800, height=600)
# fig.update_layout(margin=dict(l=20, r=20, t=100, b=5))
fig.show()

In [ ]:
date_parser = {
    "StudyStartDate": {
        "format": r"%y/%m/%d"
    },
    "StudyCompletionDate": {
        "format": r"%y/%m/%d"
    },
    "LastUpdatePostDate": {
        "format": r"%y/%m/%d"
    }
}

conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')
meshquery1 = """
SELECT 
    m.*, 
    lo.State, 
    lo.Country, 
    COUNT(lo.State) as StateCount
FROM
    CTGMain as m
LEFT JOIN
    CTGMeshInterventions as mi
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
LEFT JOIN
    CTGLocations as lo
    ON m.nctID=lo.nctID
WHERE
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    StudyStartDate >= '2020-01-01'
AND
    InterventionType='drug'
GROUP BY
    lo.State
ORDER BY
    StateCount desc;
"""
state_df = pd.read_sql_query(meshquery1, conn, parse_dates=date_parser)
conn.close()

print(state_df)


           nctID                                         BriefTitle  Acronym  \
0    nct05580107           mdpk67b in patients with prostate cancer     null   
1    nct04381832  adenosine receptor antagonist combination ther...     null   
2    nct05458544  [lu-177]ludotadipep in castration-resistant pr...   lucida   
3    nct05458544  [lu-177]ludotadipep in castration-resistant pr...   lucida   
4    nct05053152  testing the addition of the drug relugolix to ...     null   
..           ...                                                ...      ...   
581  nct05794906  a study to compare darolutamide given with and...  arastep   
582  nct05973149  trial of qlh12016 in patients with metastatic ...     null   
583  nct05939414  an open-label study comparing lutetium (177lu)...  psma-dc   
584  nct05794906  a study to compare darolutamide given with and...  arastep   
585  nct06827080  study on the safety, tolerability, and prelimi...     null   

                           OrgName   In

In [ ]:
fig = px.scatter_geo(state_df, locations='State', locationmode='USA-states', color='StateCount', size='StateCount', scope='usa', width=1600, height=800)
fig.show()

In [ ]:

conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')
meshquery1 = """
SELECT 
    mi.MeshIntervention, 
    SUM(m.EnrollmentCount) as EnrollmentTotal, 
    COUNT(mi.MeshIntervention) as MeshInterventionCount, 
    SUM(m.NumDeaths) as TotalDeaths, 
    ROUND(SUM(CASE WHEN m.NumDeaths)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentDeaths,
    ROUND(SUM(m.NumAffectedSeriousEvent)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentSeriousEvents,
    ROUND(SUM(m.NumAffectedOtherEvent)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentOtherEvents
FROM
    CTGMain as m
LEFT JOIN
    CTGMeshInterventions as mi
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
LEFT JOIN
    CTGLocations as lo
    ON m.nctID=lo.nctID
WHERE
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    StudyStartDate >= '2020-01-01'
AND
    InterventionType='drug'
AND
    EnrollmentType='actual'
GROUP BY
    mi.MeshIntervention
ORDER BY
    MeshInterventionCount desc;
"""
outcomes_df = pd.read_sql_query(meshquery1, conn, parse_dates=date_parser)
conn.close()

print(outcomes_df)


                                      MeshIntervention  EnrollmentTotal  \
0                                           leuprolide         16826546   
1                                            goserelin         16741143   
2    acetyl-2-naphthylalanyl-3-chlorophenylalanyl-1...         16724809   
3                                          apalutamide         16863890   
4                                  triptorelin pamoate         16601466   
..                                                 ...              ...   
137                          blood specimen collection               44   
138                                         belzutifan                0   
139                                  68ga-dkfz-psma-11              163   
140  2-(3-(1-carboxy-5-((6-fluoropyridine-3-carbony...                0   
141  11-(2-pyrrolidin-1-ylethoxy)-14,19-dioxa-5,7,2...                6   

     MeshInterventionCount  TotalDeaths  PercentEnrollmentDeaths  \
0                     9341     

In [52]:
outcomes_df['MeshIntervention'] = outcomes_df['MeshIntervention'].astype(str).apply(lambda x: x if len(x) <= 20 else x[:20] + '...')
fig = px.bar(outcomes_df[:10], x='MeshIntervention', y='EnrollmentTotal', 
             hover_data=['EnrollmentTotal'],
             orientation='v', width=800, height=600)

fig.show()

In [33]:
fig = px.bar(outcomes_df[:10], x='MeshIntervention', y=['PercentEnrollmentDeaths', 'PercentEnrollmentSeriousEvents', 'PercentEnrollmentOtherEvents'],
             hover_data=['MeshInterventionCount', 'EnrollmentTotal', 'TotalDeaths'], orientation='v', width=1200, height=600, barmode='group')
fig.show()

In [13]:
date_parser = {
    "StudyStartDate": {
        "format": r"%y/%m/%d"
    },
    "StudyCompletionDate": {
        "format": r"%y/%m/%d"
    },
    "LastUpdatePostDate": {
        "format": r"%y/%m/%d"
    }
}

conn = sqlite3.connect(r'C:\Users\meccas\Databases\CTG_Prostate.db')
meshquery2 = """
SELECT
    *
FROM
    CTGMain as m
LEFT JOIN
    CTGMeshInterventions as mi
    ON m.nctID=mi.nctID
LEFT JOIN
    CTGInterventions as i
    ON m.nctID=i.nctID
LEFT JOIN
    CTGLocations as lo
    ON m.nctID=lo.nctID
WHERE
    StudyType='interventional'
AND
    PrimaryPurpose='treatment'
AND
    StudyStartDate >= '2020-01-01'
AND
    InterventionType='drug'
AND
    EnrollmentType='actual'
GROUP BY
    m.nctID
ORDER BY
    m.EnrollmentCount desc;
"""
outcomes_heatmap_df = pd.read_sql_query(meshquery2, conn, parse_dates=date_parser)
conn.close()

print(outcomes_heatmap_df)

    # mi.MeshIntervention, 
    # SUM(m.EnrollmentCount) as EnrollmentTotal, 
    # COUNT(mi.MeshIntervention) as MeshInterventionCount, 
    # SUM(m.NumDeaths) as TotalDeaths, 
    # ROUND(SUM(m.NumDeaths)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentDeaths,
    # ROUND(SUM(m.NumAffectedSeriousEvent)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentSeriousEvents,
    # ROUND(SUM(m.NumAffectedOtherEvent)/SUM(m.EnrollmentCount)*100, 2) as PercentEnrollmentOtherEvents

           nctID                                         BriefTitle  \
0    nct04513717  two studies for patients with high risk prosta...   
1    nct04191096  efficacy and safety of pembrolizumab (mk-3475)...   
2    nct04720157  an international prospective open-label, rando...   
3    nct04136353  darolutamide augments standard therapy for loc...   
4    nct05348577  study of capivasertib + docetaxel vs placebo +...   
..           ...                                                ...   
200  nct04225299  evaluation of efficacy of tookad® (vtp) versus...   
201  nct03709550  enzalutamide and decitabine in treating patien...   
202  nct03488810  apalutamide with radiotherapy and androgen dep...   
203  nct02864784  investigating the safety, tolerability and eff...   
204  nct02811809  apalutamide plus intermittent hormone therapy ...   

          Acronym                                            OrgName  \
0            null                                       nrg oncology   
1  